In [1]:
import os
import json
import pandas as pd
from moviepy.editor import VideoFileClip
from tqdm import tqdm

In [2]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]

In [3]:
#input json
#set medical video download folder
tvqa_folder = "/home/hlpark/shared/TVQA/video/video_files/"
if not os.path.exists(tvqa_folder):
    os.makedirs(tvqa_folder)

eval_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/evaluation/tvqa"
if not os.path.exists(eval_folder):
    os.makedirs(eval_folder)

vid_json_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa"
if not os.path.exists(vid_json_folder):
    os.makedirs(vid_json_folder)

In [4]:
#video duration json file
vid_duration_json = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/video_duration.json"

In [4]:
#read all video duration
video_duration_dict = {}
for root, dir, files in os.walk(os.path.join(tvqa_folder)):
     for f in tqdm(files):
        video_path = os.path.join(tvqa_folder, f)
        if video_path.endswith(".mp4"):
            video = VideoFileClip(video_path)
        else:
            video = VideoFileClip(video_path + ".mp4")
        v_duration = video.duration
        video_duration_dict[f] = v_duration

100%|██████████| 21829/21829 [26:03<00:00, 13.96it/s]


In [6]:
save_json(video_duration_dict, vid_duration_json)

In [5]:
video_duration_dict = load_jsonl(vid_duration_json)[0]

In [6]:
train_path = '/home/hlpark/shared/TVQA/tvqa_qa_release/tvqa_train.jsonl'
val_path = '/home/hlpark/shared/TVQA/tvqa_qa_release/tvqa_val.jsonl'
test_path = '/home/hlpark/shared/TVQA/tvqa_qa_release/tvqa_test_public.jsonl'

In [7]:
train = load_jsonl(train_path)
val = load_jsonl(val_path)
test = load_jsonl(test_path)

In [8]:
print(len(test))

7623


In [10]:
txt_file_root_path = "/home/hlpark/shared/TVQA"

In [12]:
new_train = {}
new_val = {}
new_test = {}
video_names = {}
vid_dict_train, vid_dict_val, vid_dict_test = {}, {}, {}
unav_train_query_cnt, unav_val_query_cnt, unav_test_query_cnt = 0, 0, 0
with open(os.path.join(txt_file_root_path, "tvqa_test_queries.txt"), "w") as f:
    for i, qa in enumerate(test):
        qa_dict = {}
        video_id = qa['vid_name'] + ".mp4"
        qa_dict[video_id] = {}
        qa_dict[video_id]['relevant'] = True
        qa_dict[video_id]['clip'] = True
        qa_dict[video_id]['bounds'] = [qa['ts'].split('-')[0], qa['ts'].split('-')[1]]
        qa_dict[video_id]['steps'] = []
        qa_dict['qid'] = qa['qid']
        
        qa_dict[video_id]['v_duration'] = video_duration_dict[video_id]
        new_test[qa['q']] = qa_dict
        f.writelines(qa['q'] + "\n")
with open(os.path.join(txt_file_root_path, "tvqa_val_queries.txt"), "w") as f:
    for i, qa in enumerate(val):
        qa_dict = {}
        video_id = qa['vid_name'] + ".mp4"
        qa_dict[video_id] = {}
        qa_dict[video_id]['relevant'] = True
        qa_dict[video_id]['clip'] = True
        qa_dict[video_id]['bounds'] = [qa['ts'].split('-')[0], qa['ts'].split('-')[1]]
        qa_dict[video_id]['steps'] = []
        qa_dict['qid'] = qa['qid']

        qa_dict[video_id]['v_duration'] = video_duration_dict[video_id]
        f.writelines(qa['q'] + "\n")

In [13]:
save_json(new_val, f'{vid_json_folder}/all_data_val.json')
save_json(new_test, f'{vid_json_folder}/all_data_test.json')

In [5]:
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    return data

In [6]:
test_json = load_result_json(f'{vid_json_folder}/all_data_test.json')